In [ ]:
!conda install tensorflow

In [ ]:
pip install tensorflow

In [ ]:
import os
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/Github/dus_mm/')
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import utils
import glob
from matplotlib import pyplot as plt
import seaborn as sb
os.chdir('data')

columnsX = ['freq','AX','BX','BY','CY','CX','DX','DY','AY']
columnsY = ['AX','BX','BY','CY','CX','DX','DY','AY','AQ','AL','BQ','BL','CQ','CL','DQ','DL']
columnsYout = ['AQ','AL','BQ','BL','CQ','CL','DQ','DL','X','Y','Q','L']

In [ ]:
X = pd.DataFrame()
y = pd.DataFrame()
numb_files = len(glob.glob('data_output*'))
for i in glob.glob('data_output*'): 
    numb = i[11]
    if (numb == 'data_output.txt'):
        numb = ''
    y = pd.concat([y,pd.read_table(os.path.join('data_output%s.txt' % numb), header = 0, names = columnsY, 
                                   index_col = False, sep='\s+', engine='python', dtype ='float')], ignore_index = 1)
    X = pd.concat([X,pd.read_table(os.path.join('data_input%s.txt' % numb), header = 0, names = columnsX, 
                                   index_col = False, sep='\s+', engine='python', dtype ="float")], ignore_index = 1)
drop_ind = y.loc[(y > 28).any(axis = 1)].index
X = utils.init_in(X)
y = utils.decomposition(y)

drop_ind.append(y.loc[(y == 0).all(axis = 1)].index)
X = X.drop(drop_ind).reset_index(drop=True)
y = y.drop(drop_ind).reset_index(drop=True)

In [ ]:
deltaf = y['X'] + y['Y'] + y['Q'] + y['L']
deltaf.name ='deltaf'
y  = y.drop(columns = ['X','Y','Q','L'])
y = pd.concat([y,deltaf],axis = 1)

In [ ]:
ind0 = y['deltaf'][y['deltaf'] == 0].index
X = X.drop(ind0).reset_index(drop=True)
y = y.drop(ind0).reset_index(drop=True)

In [ ]:
train_examples = pd.concat([X,y],axis = 1)
C_mat = train_examples.corr()
fig = plt.figure(figsize = (8,8))
#X1 = X.drop(columns = ['freq3'])
sb.heatmap(C_mat, vmax = .8, square = True)

In [ ]:
X.hist(figsize = (12,10))
plt.show()

In [ ]:
number_train = round(len(X)*0.95)
X_train = X.loc[:number_train]
X_test = X.loc[number_train:]
X_test = X_test.reset_index(drop=True)

y_train = y.loc[:number_train]
y_test = y.loc[number_train:]
y_test = y_test.reset_index(drop=True)
input_layer = X_test.shape[1]
hidden_layer = 15
output_layer = y.shape[1]

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = 18, activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(12, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer='nadam', metrics=['mean_squared_error'])
NN_model.summary()

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
y

In [ ]:
NN_model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

In [ ]:
wights_file = 'Weights-466--0.88581.hdf5'
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_squared_error', optimizer='nadam', metrics=['mean_squared_error'])

In [ ]:
predictions = NN_model.predict(X_test)
yy = pd.DataFrame(np.round(predictions), columns = y_test.columns)

In [ ]:
y_test

In [ ]:
yy - y_test

In [ ]:
yy - y_test

In [ ]:
yy - y_test

In [ ]:
os.chdir('/content/drive/MyDrive/Github/dus_mm/data/')
XX = pd.read_table(os.path.join('data.txt'), header = 0, names = columnsX, index_col = False, sep='\s+', engine='python', dtype ="float")
XX = utils.init_in(XX)
predictions = NN_model.predict(XX)
yy = np.abs(pd.DataFrame(predictions.astype('float'), columns = columnsYout))
#yy = utils.integration(yy,0.72*1.33,1.33) # перемычки 30 мкм вместо 40 мкм 1.33

In [ ]:
yy = np.abs(pd.DataFrame(np.round(predictions,0), columns = columnsYout))

In [ ]:
yy = utils.decomposition(yy)

In [ ]:
XX

In [ ]:
XX

In [ ]:
np.round(0.72*yy*1.33)

In [ ]:
np.round(yy*1.33)

In [ ]:
np.round(yy)

In [ ]:
np.round(yy)

In [ ]:
yy.round()

In [ ]:
yy.iloc[:,:8] = 2*yy.iloc[:,:8]

In [ ]:
yy.round(2)

In [ ]:
yy = utils.integration(np.round(yy))

In [ ]:
yyy = [8,2,2,20,21,13,5,13,7,4,1,9,8,22,13,17]
yyy = pd.DataFrame([yyy], columns = columnsY,  dtype ="float")
yyy = utils.decomposition(yyy)
yyy = utils.integration(yyy)

In [ ]:
yy

In [ ]:
yy = np.abs(pd.DataFrame(np.round(predictions,1), columns = columnsYout))

In [ ]:
yy = utils.integration(predictions, 0.73)

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X_train, y_train, test_size = 0.25, random_state = 14)

In [ ]:
model = RandomForestRegressor()
model.fit(train_X,train_y)

# Get the mean absolute error on the validation data
predicted = model.predict(val_X)
MAE = mean_absolute_error(val_y , predicted)
print('Random forest validation MAE = ', MAE)

In [ ]:
help(XGBRegressor)

In [ ]:
XGBModel = XGBRegressor()
XGBModel.fit(train_X,train_y , verbose=False)

# Get the mean absolute error on the validation data :
XGBpredictions = XGBModel.predict(val_X)
MAE = mean_absolute_error(val_y , XGBpredictions)
print('XGBoost validation MAE = ',MAE)

In [ ]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
#X, y = make_regression(n_samples=10, n_targets=3, random_state=1)
MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(X_train, y_train).predict(X_test)

In [ ]:
yy = pd.DataFrame(np.round(_),columns = y_test.columns)

In [ ]:
y_test

In [ ]:
yy

In [ ]:
y_test - yy